In [77]:
import os as os
from pathlib import Path
import numpy as np
import pandas as pd
# import polars as pl
import tqdm
# import slippi as slp=
from joblib import Parallel, delayed
from multiprocessing import Manager
import pyarrow as pa
# import pyarrow.parquet as pq
import gzip
import pickle
# import feather
import uuid
import sys
import time
sys.path.append('../..')
from slp.slp_package.slp_functions import one_hot_encode_flags
import matplotlib.pyplot as plt



In [78]:
# # Process Mango's dataframe and save it.
# df = pd.read_parquet("C:\\Users\\jaspa\\Grant ML\\slp\\data\\mango_all_game_data_df.parquet")
# # print(full_df.columns)

# df_r = df.loc[df['num_players'] == 2]
# df_r = df_r.loc[df_r['is_teams'] == False]
# df_r = df_r.loc[df_r['conclusive'] == True]
# df_r = df_r.loc[df_r['player_1_type_name'] == 'HUMAN']
# df_r = df_r.loc[df_r['player_2_type_name'] == 'HUMAN']
# # Define the list of stage names to include
# stages_to_include = [
#     'BATTLEFIELD',
#     'YOSHIS_STORY',
#     'DREAM_LAND_N64',
#     'POKEMON_STADIUM',
#     'FINAL_DESTINATION',
#     'FOUNTAIN_OF_DREAMS'
# ]

# # Filter the dataframe for the desired stages
# df_r = df_r[df_r['stage_name'].isin(stages_to_include)]

# # Filter to include only games where MANG#0 is playing
# df_mango = df_r[(df_r['player_1_netplay_code'] == 'MANG#0') | (df_r['player_2_netplay_code'] == 'MANG#0')]

# # Create the new dataframe
# df_mango_stats = df_mango.apply(
#     lambda row: pd.Series({
#         'mango_character_name': row['player_1_character_name'] if row['player_1_netplay_code'] == 'MANG#0' else row['player_2_character_name'],
#         'opponent_netplay_code': row['player_1_netplay_code'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_netplay_code'],
#         'opponent_netplay_name': row['player_1_netplay_name'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_netplay_name'],
#         'opponent_character_name': row['player_1_character_name'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_character_name'],
#         'stage_name': row['stage_name'],
#         'mango_win': row['player_1_win'] if row['player_1_netplay_code'] == 'MANG#0' else row['player_2_win'],
#         'date': row['date'],
#         'duration': row['duration'],
#         'mango_all_data_df_path': row['player_1_all_data_df_save_path'] if row['player_1_netplay_code'] == 'MANG#0' else row['player_2_all_data_df_save_path'],
#         'mango_inputs_df': row['player_1_inputs_df_save_path'] if row['player_1_netplay_code'] == 'MANG#0' else row['player_2_inputs_df_save_path'],
#         'mango_inputs_np': row['player_1_inputs_np_save_path'] if row['player_1_netplay_code'] == 'MANG#0' else row['player_2_inputs_np_save_path'],
#         'opponent_all_data_df_path': row['player_1_all_data_df_save_path'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_all_data_df_save_path'],
#         'opponent_inputs_df': row['player_1_inputs_df_save_path'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_inputs_df_save_path'],
#         'opponent_inputs_np': row['player_1_inputs_np_save_path'] if row['player_2_netplay_code'] == 'MANG#0' else row['player_2_inputs_np_save_path'],
#         'source_path_suffix': row['source_path_suffix']
#     }), axis=1
# )
# # Normalize the opponent_netplay_code
# df_mango_stats['normalized_netplay_codes'] = df_mango_stats['opponent_netplay_code'].apply(lambda x: x.split('#')[0])
# df_mango_stats.head()
# df_mango_stats.to_parquet("C:\\Users\\jaspa\\Grant ML\\slp\\data\\processed_mango_df.parquet", engine = 'pyarrow', compression='gzip')

<h2> Import Mango's Processed Data </h2>
We import the dataframe we processed above. We have filtered so that only two player games played on competitive stages where the winner was determined remain. We have changed the dataframe so that we have mango and the opponent rather than player 1 and player 2.

In this part we also load dataframes and print column values and column names so that we know what is in them when we work with them later in the file.

In [79]:
all_games_df = pd.read_parquet("C:\\Users\\jaspa\\Grant ML\\slp\\data\\processed_mango_df.parquet", engine = 'pyarrow')
falco_df = all_games_df[all_games_df['mango_character_name'] == 'FALCO']
fox_df = all_games_df[all_games_df['mango_character_name'] == 'FOX']
print(all_games_df.columns)


Index(['mango_character_name', 'opponent_netplay_code',
       'opponent_netplay_name', 'opponent_character_name', 'stage_name',
       'mango_win', 'date', 'duration', 'mango_all_data_df_path',
       'mango_inputs_df', 'mango_inputs_np', 'opponent_all_data_df_path',
       'opponent_inputs_df', 'opponent_inputs_np', 'source_path_suffix',
       'normalized_netplay_codes'],
      dtype='object')


In [80]:
print(all_games_df['stage_name'].value_counts())

stage_name
BATTLEFIELD           6482
YOSHIS_STORY          4882
POKEMON_STADIUM       4720
DREAM_LAND_N64        4624
FOUNTAIN_OF_DREAMS    4201
FINAL_DESTINATION     4092
Name: count, dtype: int64


In [81]:
file_path = falco_df[all_games_df['stage_name'] == 'BATTLEFIELD'].iloc[0]['mango_all_data_df_path']
all_frame_data_df = pd.read_parquet(file_path, engine = 'pyarrow')
print(all_frame_data_df.columns)

Index(['frame_index', 'pre_state', 'pre_position_x', 'pre_position_y',
       'pre_direction', 'pre_joystick_x', 'pre_joystick_y', 'pre_cstick_x',
       'pre_cstick_y', 'pre_triggers_logical', 'pre_triggers_physical_l',
       'pre_triggers_physical_r', 'pre_buttons_logical',
       'pre_buttons_physical', 'pre_random_seed', 'pre_raw_analog_x',
       'pre_damage', 'post_character', 'post_state', 'post_position_x',
       'post_position_y', 'post_direction', 'post_damage', 'post_sheild',
       'post_stocks', 'post_last_attack_landed', 'post_last_hit_by',
       'post_combo_count', 'post_state_age', 'post_flags', 'post_hit_stun',
       'post_airbourn', 'post_ground', 'post_jumps', 'post_l_cancel'],
      dtype='object')


C:\Users\jaspa\AppData\Local\Temp\ipykernel_32952\2875622874.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  file_path = falco_df[all_games_df['stage_name'] == 'BATTLEFIELD'].iloc[0]['mango_all_data_df_path']


<h2> Find Boundry of Stages </h2>
We import several games played on each stage and find the max and min of both the x and y coordinates of the position of the player as well as some other information about the size of the stages.

Results: The dimension of the stages and their aspect ratios are all different.

In [82]:
def player_position_limits(file_path, x_min_list, x_max_list, y_min_list, y_max_list):
    all_frame_data_df = pd.read_parquet(file_path)#, engine = 'pyarrow')
    x_min_list.append(all_frame_data_df['pre_position_x'].min())
    x_max_list.append(all_frame_data_df['pre_position_x'].max())
    y_min_list.append(all_frame_data_df['pre_position_y'].min())
    y_max_list.append(all_frame_data_df['pre_position_y'].max())
    return

def stage_limits(stage, df, sample_size):
    # Make a list of paths to the frame data dataframes of games played on the stage of interest
    stage_df = df[df['stage_name'] == stage]
    file_paths_list = stage_df['mango_all_data_df_path'].to_list()
    file_paths_list.append(stage_df['mango_all_data_df_path'].to_list())
    
    # Create shared lists
    manager = Manager()
    x_min_list = manager.list()
    x_max_list = manager.list()
    y_min_list = manager.list()
    y_max_list = manager.list()
    
    # Process the files in parallel using joblib
    Parallel(n_jobs=20, verbose=0)(delayed(player_position_limits)(file_path, x_min_list, x_max_list, y_min_list, y_max_list) for file_path in tqdm.tqdm(file_paths_list[:sample_size]))
    
    # Find the max and min of the list we created
    x_min = min(x_min_list)
    x_max = max(x_max_list)
    y_min = min(y_min_list)
    y_max = max(y_max_list)
    
    return x_min, x_max, y_min, y_max


In [84]:
# Find the boundries of the competitive stages
sample_size = -1
data = []
for stage in all_games_df['stage_name'].unique():
    x_min, x_max, y_min, y_max = stage_limits(stage, all_games_df, sample_size)
    data.append([stage, x_min, x_max, y_min, y_max])
    
# Make a dataframe with the information we are interested in
stage_boundries_df = pd.DataFrame(data, columns = ['stage_name', 'x_min', 'x_max', 'y_min', 'y_max'])
stage_boundries_df['x_length'] = stage_boundries_df['x_max']-stage_boundries_df['x_min']
stage_boundries_df['y_length'] = stage_boundries_df['y_max']-stage_boundries_df['y_min']
stage_boundries_df['aspect_ratio'] = stage_boundries_df['x_length'] / stage_boundries_df['y_length']
stage_boundries_df['area'] = stage_boundries_df['x_length'] * stage_boundries_df['y_length']

# Print the dataframe
stage_boundries_df

    

100%|██████████| 4624/4624 [00:02<00:00, 1733.97it/s]


,stage_name,x_min,x_max,y_min,y_max,x_length,y_length,aspect_ratio,area
0,FINAL_DESTINATION,-262.615997,262.358795,-147.122879,249.358032,524.974792,396.480911,1.324086,208142.484108
1,POKEMON_STADIUM,-237.213669,246.358109,-118.742371,229.559525,483.571777,348.301895,1.388370,168428.966486
2,FOUNTAIN_OF_DREAMS,-215.184662,215.615021,-155.084824,247.430847,430.799683,402.515671,1.070268,173403.623219
3,BATTLEFIELD,-239.080933,242.023544,-116.503189,303.758179,481.104477,420.261368,1.144774,202189.625528
4,YOSHIS_STORY,-180.917068,181.560318,-98.480011,235.927185,362.477386,334.407196,1.083940,121215.046441
5,DREAM_LAND_N64,-268.762604,270.661133,-130.667969,326.973755,539.423737,457.641724,1.178703,246862.808573
